In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from sklearn.metrics import make_scorer

In [2]:
#pip install xlrd

In [3]:
# loading dataset
df= pd.read_excel("TrainDataset2023.xls")
df.head()

,ID,pCR (outcome),RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,TRG002174,1,144.0,41.0,0,0,0,1,3,3,...,0.517172,0.375126,3.325332,0.002314,3880771.500,473.464852,0.000768,0.182615,0.030508,0.000758
1,TRG002178,0,142.0,39.0,1,1,0,0,3,3,...,0.444391,0.444391,3.032144,0.005612,2372009.744,59.459710,0.004383,0.032012,0.001006,0.003685
2,TRG002204,1,135.0,31.0,0,0,0,1,2,1,...,0.534549,0.534549,2.485848,0.006752,1540027.421,33.935384,0.007584,0.024062,0.000529,0.006447
3,TRG002206,0,12.0,35.0,0,0,0,1,3,3,...,0.506185,0.506185,2.606255,0.003755,6936740.794,46.859265,0.005424,0.013707,0.000178,0.004543
4,TRG002210,0,109.0,61.0,1,0,0,0,2,1,...,0.462282,0.462282,2.809279,0.006521,1265399.054,39.621023,0.006585,0.034148,0.001083,0.005626


In [4]:
df = df.rename(columns={"pCR (outcome)": "PCR"})

In [5]:
df.head()

,ID,PCR,RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,TRG002174,1,144.0,41.0,0,0,0,1,3,3,...,0.517172,0.375126,3.325332,0.002314,3880771.500,473.464852,0.000768,0.182615,0.030508,0.000758
1,TRG002178,0,142.0,39.0,1,1,0,0,3,3,...,0.444391,0.444391,3.032144,0.005612,2372009.744,59.459710,0.004383,0.032012,0.001006,0.003685
2,TRG002204,1,135.0,31.0,0,0,0,1,2,1,...,0.534549,0.534549,2.485848,0.006752,1540027.421,33.935384,0.007584,0.024062,0.000529,0.006447
3,TRG002206,0,12.0,35.0,0,0,0,1,3,3,...,0.506185,0.506185,2.606255,0.003755,6936740.794,46.859265,0.005424,0.013707,0.000178,0.004543
4,TRG002210,0,109.0,61.0,1,0,0,0,2,1,...,0.462282,0.462282,2.809279,0.006521,1265399.054,39.621023,0.006585,0.034148,0.001083,0.005626


In [6]:
# dropping the ID because it does not offer any unique feature to prediction
df = df.drop('ID', axis=1)

In [7]:
# replacing 99 with nan values
df.replace(999, np.nan, inplace= True)
null = df.isna().sum().sum()
print("Number of null values: ",null)

Number of null values:  17


In [8]:
# dropping the nan values
df= df.dropna()

In [9]:
# checking for any duplicate values
duplicates = df.duplicated().sum()
print('number of duplicated rows: ',duplicates)

number of duplicated rows:  0


In [10]:
# checking for any outliners

def remove_outliers(df):
    threshold = 3
    z_scores = np.abs((df - df.mean()) / df.std())
    df = df[(z_scores < threshold).all(axis=1)]
    return df


In [11]:
# removing outliners
df = remove_outliers(df)

In [12]:
df1 = df.copy()
#correlated features of dataset
def correlation(data, threshold):
    col_corr = {}  # Dictionary to store correlated features
    corr_matrix = data.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:  # We are interested in absolute coefficient value
                colname = corr_matrix.columns[i]
                if colname not in col_corr:
                    col_corr[colname] = set()
                col_corr[colname].add(corr_matrix.columns[j])

    return col_corr

corr_features = correlation(df1, 0.8)
print('correlated features: ', len(corr_features))

correlated features:  87


In [13]:
# removing the correlated features
df_corr= df1.drop(labels=corr_features, axis=1)
df_corr.shape

(304, 32)

In [14]:
# dataset using correlated feature selection
X_corr = df_corr.drop(["PCR","RelapseFreeSurvival (outcome)"],axis=1)
y_corr = df_corr["RelapseFreeSurvival (outcome)"]

In [15]:
X_corr.head()

,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,HistologyType,LNStatus,TumourStage,original_shape_Elongation,...,original_gldm_DependenceEntropy,original_gldm_SmallDependenceEmphasis,original_glrlm_RunLengthNonUniformity,original_glrlm_ShortRunHighGrayLevelEmphasis,original_glszm_GrayLevelNonUniformity,original_glszm_GrayLevelNonUniformityNormalized,original_glszm_LargeAreaEmphasis,original_glszm_SizeZoneNonUniformityNormalized,original_glszm_SmallAreaEmphasis,original_ngtdm_Coarseness
2,31.0,0,0.0,0.0,1.0,2.0,1.0,0.0,2,0.645083,...,2.863091,0.007181,114.255050,0.503046,72.027027,0.973338,1561963.432,0.275749,0.534549,0.007584
3,35.0,0,0.0,0.0,1.0,3.0,1.0,1.0,3,0.770842,...,2.756432,0.004902,212.625994,0.399896,99.019802,0.980394,7007670.723,0.253014,0.506185,0.005424
4,61.0,1,0.0,0.0,0.0,2.0,1.0,0.0,2,0.861035,...,2.676170,0.007222,101.119011,0.473278,56.034483,0.966112,1288913.690,0.216409,0.462282,0.006585
5,49.0,0,0.0,0.0,1.0,3.0,1.0,1.0,3,0.948285,...,2.930247,0.008891,236.880022,0.501885,98.020000,0.980200,1815164.550,0.272800,0.532369,0.002652
7,41.0,0,0.0,0.0,1.0,3.0,1.0,1.0,2,0.951395,...,3.358576,0.010091,226.647909,0.558101,86.022727,0.977531,1045481.284,0.218492,0.476326,0.002821


In [85]:
# saving the feature in text file for future use in model training and prediction
with open('30corRFS.txt', 'w') as f:
    for feature in X_corr.columns:
        f.write(feature + '\n')


In [16]:
# feature selection for correlated set
X_for = df_corr.drop(["PCR","RelapseFreeSurvival (outcome)"],axis=1)
y_for = df_corr["RelapseFreeSurvival (outcome)"]

In [63]:
# forward feature selection
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SequentialFeatureSelector

# Creating a SequentialFeatureSelector object with RandomForestClassifier as the estimator
selector = SequentialFeatureSelector(estimator=RandomForestRegressor(n_estimators=100), n_features_to_select='auto')

# Fitting the selector to the data
selector.fit(X_for, y_for)

# Getting the selected features
selected_features = X_for.columns[selector.get_support()]

In [64]:
# Creating a copy of the dataset with the selected features
X_selected_for = df_corr[selected_features]
X_selected_for.info()


<class 'pandas.core.frame.DataFrame'>
Index: 304 entries, 2 to 399
Data columns (total 15 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   ER                                              304 non-null    int64  
 1   PgR                                             304 non-null    float64
 2   HER2                                            304 non-null    float64
 3   TrippleNegative                                 304 non-null    float64
 4   ChemoGrade                                      304 non-null    float64
 5   HistologyType                                   304 non-null    float64
 6   LNStatus                                        304 non-null    float64
 7   original_shape_Flatness                         304 non-null    float64
 8   original_firstorder_10Percentile                304 non-null    float64
 9   original_firstorder_90Percentile                

In [66]:
# Saving the forward selected features to a text file
with open('selected_features_forwardRFS.txt', 'w') as f:
    for feature in selected_features:
        f.write(feature + '\n')

In [17]:
# loading the text file
with open('selected_features_forwardRFS.txt', 'r') as f:
    selected_features = []
    for line in f:
        selected_features.append(line.strip())

In [18]:
# dataset using correlated feature selection for forward feature selection
df3 = df.copy()
X2 = df3[selected_features]
y2 = df3["RelapseFreeSurvival (outcome)"]

In [19]:
X2.shape

(304, 15)

In [20]:
# Split the data into training and testing sets for correlated features
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_corr, y_corr, test_size=0.2,shuffle=True,random_state=60)
# Standardize the features
scaler = StandardScaler()
X_train_sc1 = scaler.fit_transform(X_train1)
X_test_sc1 = scaler.fit_transform(X_test1)


In [21]:
# Split the data into training and testing sets for forward feature selection
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2,shuffle=True,random_state=60)
# Standardize the features
scaler = StandardScaler()
X_train_sc2 = scaler.fit_transform(X_train2)
X_test_sc2 = scaler.fit_transform(X_test2)


In [21]:

# Defining hyperparameter grid for LinearRegression
param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [1, 2, 4, 8],
    
    
}

# Defining MSE and MAE scorers
mse_scorer = mean_squared_error
mae_scorer = mean_absolute_error

# Instantiating LinearRegression
model1 = LinearRegression()

# Creating GridSearchCV objects
grid_search1 = GridSearchCV(estimator=model1, param_grid=param_grid, scoring=mse_scorer, cv=5)
grid_search2 = GridSearchCV(estimator=model1, param_grid=param_grid, scoring=mae_scorer, cv=5)

# Fitting the GridSearchCV objects 
grid_search1.fit(X_train_sc1, y_train1)
grid_search2.fit(X_train_sc2, y_train2)

# Getting the best parameters from GridSearchCV 
best_params1 = grid_search1.best_params_
best_params2 = grid_search2.best_params_

# Printting the best parameters for each dataset
print("correlation:", best_params1)
print("Forward:", best_params2)

# Getting the MSE and MAE for the best model from GridSearchCV for each dataset
best_model1 = LinearRegression(**best_params1)
best_model1.fit(X_train_sc1, y_train1)
y_pred1 = best_model1.predict(X_test_sc1)
mse1 = mean_squared_error(y_test1, y_pred1)
mae1 = mean_absolute_error(y_test1, y_pred1)

best_model2 = LinearRegression(**best_params2)
best_model2.fit(X_train_sc2, y_train2)
y_pred2 = best_model2.predict(X_test_sc2)
mse2 = mean_squared_error(y_test2, y_pred2)
mae2 = mean_absolute_error(y_test2, y_pred2)

# Printting the MSE and MAE for each dataset
print("MSE for correlation:", mse1)
print("MAE for correlation:", mae1)
print("MSE for Forward:", mse2)
print("MAE for Forward:", mae2)


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 192, in wrapper
    params = func_sig.bind(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\inspect.py", line 3212, in bind
    return self._bind(args, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\inspect.py", line 3138, in _bind
    raise TypeError(
TypeError: too many positional arguments

  warnings.warn(
C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this tra

correlation: {'copy_X': True, 'fit_intercept': True, 'n_jobs': 1}
Forward: {'copy_X': True, 'fit_intercept': True, 'n_jobs': 1}
MSE for correlation: 799.3026001696544
MAE for correlation: 23.451480894247755
MSE for Forward: 694.2268453300169
MAE for Forward: 22.10205989015688


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 192, in wrapper
    params = func_sig.bind(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\inspect.py", line 3212, in bind
    return self._bind(args, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\inspect.py", line 3138, in _bind
    raise TypeError(
TypeError: too many positional arguments

  warnings.warn(
C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this tra

In [21]:
# Defining a grid of parameters for SVR
param_grid = {
  'kernel': ['linear', 'rbf', 'poly'],
  'C': [0.1, 1, 10],
  'gamma': [0.01, 0.1, 1],
  'epsilon': [0.1, 0.5, 1.0]
}

# Performming SVM regression grid search for correlation dataset
correlation_svr = SVR()
correlation_grid_search = GridSearchCV(correlation_svr, param_grid, scoring="neg_mean_squared_error")
correlation_grid_search.fit(X_train_sc1, y_train1)

# Getting the best parameters for correlation dataset
correlation_best_params = correlation_grid_search.best_params_

# Calculating MSE and MAE for correlation dataset
correlation_mse = mean_squared_error(correlation_grid_search.predict(X_test_sc1), y_test1)
correlation_mae = mean_absolute_error(correlation_grid_search.predict(X_test_sc1), y_test1)

# Performming SVM regression grid search for forward dataset
forward_svr = SVR()
forward_grid_search = GridSearchCV(forward_svr, param_grid, scoring="neg_mean_squared_error")
forward_grid_search.fit(X_train_sc2, y_train2)

# Getting the best parameters for forward dataset
forward_best_params = forward_grid_search.best_params_

# Calculating MSE and MAE for forward dataset
forward_mse = mean_squared_error(forward_grid_search.predict(X_test_sc2), y_test2)
forward_mae = mean_absolute_error(forward_grid_search.predict(X_test_sc2), y_test2)

# Printting the best parameters for each dataset
print("Best parameters for correlation dataset:", correlation_best_params)
print("Best parameters for forward dataset:", forward_best_params)

# Printting the MSE and MAE for each dataset
print("MSE for correlation dataset:", correlation_mse)
print("MAE for correlation dataset:", correlation_mae)
print("MSE for forward dataset:", forward_mse)
print("MAE for forward dataset:", forward_mae)

Best parameters for correlation dataset: {'C': 10, 'epsilon': 1.0, 'gamma': 0.01, 'kernel': 'rbf'}
Best parameters for forward dataset: {'C': 10, 'epsilon': 1.0, 'gamma': 0.1, 'kernel': 'rbf'}
MSE for correlation dataset: 742.8942486653511
MAE for correlation dataset: 23.802029711013077
MSE for forward dataset: 649.8472504170526
MAE for forward dataset: 21.968115173841937


In [26]:
from sklearn.tree import DecisionTreeRegressor

# Defining hyperparameter grid for DecisionTreeRegressor
param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'splitter': ['best', 'random'],
    'max_depth': [2, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5],
    'random_state':[42]
}

# Instantiating DecisionTreeRegressor 
dtr1 = DecisionTreeRegressor()
dtr2 = DecisionTreeRegressor()

# Creating GridSearchCV objects
grid_search1 = GridSearchCV(estimator=dtr1, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search2 = GridSearchCV(estimator=dtr2, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fitting the GridSearchCV objects 
grid_search1.fit(X_train_sc1, y_train1)
grid_search2.fit(X_train_sc2, y_train2)

# Getting the best parameters from GridSearchCV 
best_params1 = grid_search1.best_params_
best_params2 = grid_search2.best_params_

# Printting the best parameters for each dataset
print("correlation:\n", best_params1)
print("forward:\n", best_params2)

# Getting the MSE and MAE for the best model from GridSearchCV for each dataset
best_model1 = DecisionTreeRegressor(**best_params1)
best_model1.fit(X_train_sc1, y_train1)
y_pred1 = best_model1.predict(X_test_sc1)
mse_test1 = mse_scorer(y_test1, y_pred1)
mae_test1 = mae_scorer(y_test1, y_pred1)

best_model2 = DecisionTreeRegressor(**best_params2)
best_model2.fit(X_train_sc2, y_train2)
y_pred2 = best_model2.predict(X_test_sc2)
mse_test2 = mse_scorer(y_test2, y_pred2)
mae_test2 = mae_scorer(y_test2, y_pred2)

print("correlation MSE:", mse_test1)
print("forward MSE:", mse_test2)

print("correlation MAE:", mae_test1)
print("forward MAE:", mae_test2)





correlation:
 {'criterion': 'squared_error', 'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 42, 'splitter': 'random'}
forward:
 {'criterion': 'absolute_error', 'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 42, 'splitter': 'random'}
correlation MSE: 722.0637297853732
forward MSE: 663.3171675774134
correlation MAE: 22.452224990749176
forward MAE: 21.737704918032787


In [34]:
from sklearn.neural_network import MLPRegressor

# Defining MSE and MAE scorers
mse_scorer = mean_squared_error
mae_scorer = mean_absolute_error

# Defining hyperparameter grid for MLPRegressor
param_grid = {
    'hidden_layer_sizes': [(40,), (50,), (60,)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    
    'random_state': [0]
}

# Instantiate MLPRegressor
mlp1 = MLPRegressor(max_iter = 500)
mlp2 = MLPRegressor(max_iter = 500)

# Creating GridSearchCV objects 
grid_search1 = GridSearchCV(estimator=mlp1, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search2 = GridSearchCV(estimator=mlp2, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fitting the GridSearchCV objects 
grid_search1.fit(X_train_sc1, y_train1)
grid_search2.fit(X_train_sc2, y_train2)

# Getting the best parameters from GridSearchCV 
best_params1 = grid_search1.best_params_
best_params2 = grid_search2.best_params_

# Printting the best parameters for each dataset
print("correlation:\n", best_params1)
print("forward:\n", best_params2)


# Getting the MSE and MAE for the best model from GridSearchCV for each dataset
best_model1 = MLPRegressor(**best_params1,max_iter= 500)
best_model1.fit(X_train_sc1, y_train1)
y_pred1 = best_model1.predict(X_test_sc1)
mse_test1 = mse_scorer(y_test1, y_pred1)
mae_test1 = mae_scorer(y_test1, y_pred1)

best_model2 = MLPRegressor(**best_params2,max_iter= 500)
best_model2.fit(X_train_sc2, y_train2)
y_pred2 = best_model2.predict(X_test_sc2)
mse_test2 = mse_scorer(y_test2, y_pred2)
mae_test2 = mae_scorer(y_test2, y_pred2)

print("correlation MSE:", mse_test1)
print("forward MSE:", mse_test2)

print("correlation MAE:", mae_test1)
print("forward MAE:", mae_test2)


C:\Anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Opti

correlation:
 {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (40,), 'learning_rate': 'constant', 'random_state': 0, 'solver': 'sgd'}
forward:
 {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (40,), 'learning_rate': 'adaptive', 'random_state': 0, 'solver': 'sgd'}


C:\Anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


correlation MSE: 881.2163771835553
forward MSE: 876.9477464893921
correlation MAE: 23.870518686199063
forward MAE: 24.1832679515991


C:\Anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [24]:

# Defining hyperparameter grid for RandomForestRegressor
param_grid = {'n_estimators': [100, 200,500],
       'max_depth': [5, 10],
        'min_samples_leaf': [ 5, 10],
        'min_samples_split': [3, 5,10],
        'random_state': [0]}

# Instantiating RandomForestRegressor 
rf1 = RandomForestRegressor()
rf2 = RandomForestRegressor()

# Creating GridSearchCV objects 
grid_search1 = GridSearchCV(estimator=rf1, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search2 = GridSearchCV(estimator=rf2, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fitting the GridSearchCV objects 
grid_search1.fit(X_train_sc1, y_train1)
grid_search2.fit(X_train_sc2, y_train2)

# Getting the best models 
best_model1 = grid_search1.best_estimator_
best_model2 = grid_search2.best_estimator_

# Getting the MAE and MSE for best model from GridSearchCV 
mae_best1 = mean_absolute_error(y_test1, best_model1.predict(X_test_sc1))
mse_best1 = mean_squared_error(y_test1, best_model1.predict(X_test_sc1))

mae_best2 = mean_absolute_error(y_test2, best_model2.predict(X_test_sc2))
mse_best2 = mean_squared_error(y_test2, best_model2.predict(X_test_sc2))

# Printting the best parameters and best scores
print("Correlated :")
print("Best parameters:", grid_search1.best_params_)
print("Best score:", -grid_search1.best_score_)
print("Best MAE:", mae_best1)
print("Best MSE:", mse_best1)

print("Forward:")
print("Best parameters:", grid_search2.best_params_)
print("Best score:", -grid_search2.best_score_)
print("Best MAE:", mae_best2)
print("Best MSE:", mse_best2)


Correlated :
Best parameters: {'max_depth': 5, 'min_samples_leaf': 10, 'min_samples_split': 3, 'n_estimators': 100, 'random_state': 0}
Best score: 658.5387149712149
Best MAE: 22.40340800780245
Best MSE: 681.2316511309667
Forward:
Best parameters: {'max_depth': 5, 'min_samples_leaf': 10, 'min_samples_split': 3, 'n_estimators': 500, 'random_state': 0}
Best score: 652.9088661226073
Best MAE: 22.236350623056467
Best MSE: 698.5029707689725


In [22]:
from sklearn.ensemble import GradientBoostingRegressor

# Defining hyperparameter grid for GradientBoostingRegressor
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [5, 10],
    'min_samples_split': [5, 10],
    'loss': ['squared_error'],
    'subsample': [0.75, 1.0],
    'random_state': [42]
}

# Instantiating GradientBoostingRegressor
model = GradientBoostingRegressor()

# Creating GridSearchCV objects
grid_search1 = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search2 = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fitting the GridSearchCV objects 
grid_search1.fit(X_train_sc1, y_train1)
grid_search2.fit(X_train_sc2, y_train2)

# Getting the best models 
best_model1 = grid_search1.best_estimator_
best_model2 = grid_search2.best_estimator_

# Getting the MAE and MSE for best model from GridSearchCV for each dataset
mae_best1 = mean_absolute_error(y_test1, best_model1.predict(X_test_sc1))
mse_best1 = mean_squared_error(y_test1, best_model1.predict(X_test_sc1))

mae_best2 = mean_absolute_error(y_test2, best_model2.predict(X_test_sc2))
mse_best2 = mean_squared_error(y_test2, best_model2.predict(X_test_sc2))

# Printing the best parameters and best scores
print("Correlated :")
print("Best parameters:", grid_search1.best_params_)
print("Best score:", -grid_search1.best_score_)
print("Best MAE:", mae_best1)
print("Best MSE:", mse_best1)

print("Forward:")
print("Best parameters:", grid_search2.best_params_)
print("Best score:", -grid_search2.best_score_)
print("Best MAE:", mae_best2)
print("Best MSE:", mse_best2)


Correlated :
Best parameters: {'learning_rate': 0.05, 'loss': 'squared_error', 'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 100, 'random_state': 42, 'subsample': 0.75}
Best score: 718.4933588740963
Best MAE: 22.350649571604905
Best MSE: 713.3278382714217
Forward:
Best parameters: {'learning_rate': 0.05, 'loss': 'squared_error', 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100, 'random_state': 42, 'subsample': 1.0}
Best score: 647.3677538976447
Best MAE: 22.601121999715755
Best MSE: 729.8249392117732
